In [ ]:
import subprocess
import os
import re

# Retrieve database details from environment variables
DB_HOST = os.getenv('DB_HOST', 'partner-prod-temp.postgres.database.azure.com')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'impactpartners')
DB_USER = os.getenv('DB_USER', 'partnerproductionpsqlmanager@partner-prod-temp')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'uH1fSuHsnUnF44GD?5t/6zOQ')
DB_KEY_VALUE = os.getenv('KEY_VALUE','ee7f446b-488a-47f6-8251-f32ea069060c')

# ANSI color and style codes
class Style:
    RED = '\033[1;31m'  # Red color, bold
    GREEN = '\033[1;32m' # Green color, bold
    YELLOW = '\033[1;33m' # Yellow color, bold
    WHITE = '\033[37m'
    RESET = '\033[0m'  # Reset to default

# Print the values for debugging
print(f"\t{Style.RED}DB_HOST:{Style.RESET} {Style.GREEN}{DB_HOST}{Style.RESET}")
print(f"\t{Style.RED}DB_PORT:{Style.RESET} {Style.GREEN}{DB_PORT}{Style.RESET}")
print(f"\t{Style.RED}DB_NAME:{Style.RESET} {Style.GREEN}{DB_NAME}{Style.RESET}")
print(f"\t{Style.RED}DB_USER:{Style.RESET} {Style.GREEN}{DB_USER}{Style.RESET}")
# Be cautious with printing the password. Uncomment the next line only if absolutely necessary for debugging.
# print("DB_PASSWORD:", DB_PASSWORD)

# Path to the file containing SQL queries, adjusted to read from the /app/outputs directory
query_file_path = "/app/outputs/query_log.txt"
# Initialize a dictionary to store line counts for each table
table_line_counts = {}
# Function to extract the table name from the SQL query
# Function to extract table name from a line
def extract_table_name_from_line(line):
    if line.startswith("Table:"):
        return line.split("Table:")[1].strip()
    return None

# Read the file and execute queries
with open(query_file_path, "r") as file:
    lines = file.readlines()

# Initialize variables
current_table_name = None
query = ""
count_csvs = 0
# Iterate through each line in the file
# Iterate through each line in the file
for line in lines:
    line = line.strip()
    if not line:
        continue  # Skip empty lines

    new_table_name = extract_table_name_from_line(line)
    if new_table_name:
        if query and current_table_name:
            # Process the previous query
            command = [
                "psql",
                "-h", DB_HOST,
                "-p", str(DB_PORT),
                "-d", DB_NAME,
                "-U", DB_USER,
                "-c", f"SELECT COUNT(*) FROM ({query}) as count_result;"
            ]
            try:
                process = subprocess.Popen(command, env={"PGPASSWORD": DB_PASSWORD},
                                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                output, error = process.communicate()

                if output:
                    output_str = output.decode().strip()
                    print(f"\t{Style.WHITE}Output for{Style.RESET} {Style.YELLOW}{current_table_name}{Style.RESET}: {Style.GREEN}{output_str}{Style.RESET}")

                    # Split the output by lines and extract the count
                    lines = output_str.splitlines()
                    if len(lines) >= 3:
                        try:
                            count_str = lines[2].strip()  # The count is expected to be on the third line
                            count = int(count_str)
                            table_line_counts[current_table_name] = count
                        except ValueError:
                            print(f"\t{Style.RED}Error parsing count result for {current_table_name}:{count_str}{Style.RESET}")
                    else:
                        print(f"\t{Style.RED}Unexpected output format for {current_table_name}: {output_str}{Style.RESET}")

                if error:
                    print(f"\t{Style.RED}Error in count query for {current_table_name}: {error.decode()}{Style.RESET}")

            except Exception as e:
                print(f"\t{Style.RED}Error executing query for {current_table_name}: {e}{Style.RESET}")
            count_csvs += 1

        current_table_name = new_table_name
        query = ""
    else:
        query += line + " "

# Handle the last query in the file
if query and current_table_name:
    command = [
        "psql",
        "-h", DB_HOST,
        "-p", str(DB_PORT),
        "-d", DB_NAME,
        "-U", DB_USER,
        "-c", f"SELECT COUNT(*) FROM ({query}) as count_result;"
    ]
    try:
        process = subprocess.Popen(command, env={"PGPASSWORD": DB_PASSWORD},
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output, error = process.communicate()

        if output:
            output_str = output.decode().strip()
            print(f"\t{Style.WHITE}Output for{Style.RESET} {Style.YELLOW}{current_table_name}{Style.RESET}:{Style.GREEN}{output_str}{Style.RESET}")

            # Split the output by lines and extract the count
            lines = output_str.splitlines()
            if len(lines) >= 3:
                try:
                    count_str = lines[2].strip()  # The count is expected to be on the third line
                    count = int(count_str)
                    table_line_counts[current_table_name] = count
                except ValueError:
                    print(f"\t{Style.RED}Error parsing count result for {current_table_name}: {count_str}{Style.RESET}")
            else:
                print(f"\t{Style.RED}Unexpected output format for {current_table_name}: {output_str}{Style.RESET}")

        if error:
            print(f"\t{Style.RED}Error in count query for {current_table_name}: {error.decode()}{Style.RESET}")

    except Exception as e:
        print(f"\t{Style.RED}Error executing query for {current_table_name}: {e}{Style.RESET}")
    count_csvs += 1

sorted_table_counts = sorted(table_line_counts.items())
outputfile = f"/app/outputs/output_{DB_KEY_VALUE}.txt"

# Create a final output file listing each table with its line count
with open(outputfile, "w") as f:
    for table_name, count in sorted_table_counts:
        f.write(f"{table_name}: {count}\n")

print(f'\t{Style.WHITE}Files generated: {Style.RESET}{Style.GREEN}{outputfile}{Style.RESET} {Style.WHITE}containing tables and respective lines copied.{Style.RESET}')
print(f'\t{Style.WHITE}Total CSV files created {Style.RESET}{Style.GREEN}{count_csvs}{Style.RESET} under {Style.YELLOW}/app/outputs/output_(name of table).csv{Style.RESET}')


DB_HOST: default_host
DB_PORT: 5432
DB_NAME: default_db_name
DB_USER: default_user
DB_PASSWORD: default_password


FileNotFoundError: [Errno 2] No such file or directory: '/backups/query_log.txt'